In [1]:
!pip install kaggle
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.10
!mkdir -p ~/.kaggle
!cp "/content/drive/My Drive/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Found existing installation: kaggle 1.5.10
Uninstalling kaggle-1.5.10:
  Successfully uninstalled kaggle-1.5.10
  Using cached kaggle-1.5.10-py3-none-any.whl


In [ ]:
!kaggle competitions download -c logistics-shopee-code-league
!unzip -q logistics-shopee-code-league.zip

In [ ]:
import pandas as pd
import datetime
import math

location = ["manila","luzon","visayas","mindanao"]
holiday = [1585065600,1585497600,1585584000]
onSec = 24*60*60
firstSunday = 1578153600
sla = [(3,5,7,7),
       (5,5,7,7),
       (7,7,7,7),
       (7,7,7,7)]

df = pd.read_csv("delivery_orders_march.csv")

def getLocation(str):
  strSplited = str.split(" ")
  return strSplited[len(strSplited)-1].lower()

def fixDate(date):
  return math.floor(date/onSec) * onSec

def countSun(date):

  count = math.floor((date - firstSunday) / (7 * onSec))

  if (date - firstSunday) % (7 * onSec) == 0:
    count = count + 1

  return count

def timeDiffer(start,end):
  start = fixDate(start)
  end = fixDate(end) + onSec
  dateDiff = (end - start) / onSec - 1
  for date in holiday:
    if start <= date and date <= end:
      dateDiff = dateDiff - 1
  dateDiff = dateDiff - (countSun(end - onSec) - countSun(start))
  return dateDiff

orderid = []
is_late = []

for row in df.itertuples(index=False):
    start = getLocation(row.selleraddress)
    end = getLocation(row.buyeraddress);
    lim = sla[location.index(start)][location.index(end)]
    orderid.append(row.orderid)
    late = False

  ##  print(str(fixDate(row.pick)) + " " + str(fixDate(row._2)) + " " + timeDeffer(row.pick,row._2) + " " + str(countSun(row.pick)) + " " + str(countSun(row._2)) )

    if math.isnan(row._3):
      if timeDiffer(row.pick,row._2) > lim:
        is_late.append(1)
      else:
        is_late.append(0)
    else:
      if timeDiffer(row._2,row._3) > 3 :
        is_late.append(1)
      else :
        is_late.append(0);

sub = pd.DataFrame({"orderid":orderid,"is_late":is_late})
sub.to_csv(r'answer.csv', index = False, header=True)
!kaggle competitions submit -c logistics-shopee-code-league -f answer.csv -m "Null"


100% 41.2M/41.2M [00:12<00:00, 3.41MB/s]
Successfully submitted to [Students] Shopee Code League - Logistics

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import pathlib

dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'

tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
dataframe = pd.read_csv(csv_file)

# In the original dataset "4" indicates the pet was not adopted.
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4, 0, 1)

# Drop un-used columns.
dataframe = dataframe.drop(columns=['AdoptionSpeed', 'Description'])

train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['Age'])
  print('A batch of targets:', label_batch )
feature_columns = []
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1, 2, 3, 4, 5])
feature_columns.append(age_buckets)

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

7383 train examples
1846 validation examples
2308 test examples
Every feature: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt']
A batch of ages: tf.Tensor([ 4 16  3  1  4], shape=(5,), dtype=int64)
A batch of targets: tf.Tensor([1 1 1 1 1], shape=(5,), dtype=int64)
Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
1466/1477 [============================>.] - ETA: 0s - loss: 0.5575 - accuracy: 0.6478WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Type': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=string>, 'Age': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=int64>, 'Breed1': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=string>, 'Gender': <tf.Tensor 'ExpandDims_6:0' shape=(None, 1) dtype=string>, 'Color1': <tf.Tensor 'ExpandDims_2:0'

In [4]:
!kaggle competitions download -c student-shopee-code-league-marketing-analytics
!unzip -q student-shopee-code-league-marketing-analytics.zip

  0% 0.00/2.78M [00:00<?, ?B/s]
100% 2.78M/2.78M [00:00<00:00, 227MB/s]


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

dataset = pd.read_csv('train.csv') 
testData = pd.read_csv('test.csv')
user = pd.read_csv('users.csv').fillna(0)
dataset.head()
user.head()

train = dataset.merge(user, on = ['user_id'])
test = testData.merge(user, on = ['user_id'])

to_replace = ['Never open', 'Never checkout', 'Never login']

train.replace(to_replace = to_replace,value = -1, inplace= True)
test.replace(to_replace = to_replace,value = -1, inplace= True)

features = [
            'subject_line_length',
            'last_open_day', 
            'last_login_day', 
            'last_checkout_day',
            'open_count_last_10_days', 
            'open_count_last_30_days',
            'open_count_last_60_days', 
            'login_count_last_10_days',
            'login_count_last_30_days', 
            'login_count_last_60_days',
            'checkout_count_last_10_days', 
            'checkout_count_last_30_days',
            'checkout_count_last_60_days', 
            'attr_1',
            'attr_2', 
            'attr_3'
            ]

train_ds = train[features]
train_label = train['open_flag']
test_ds = test[features]

model = SVC(
    C = 0.01,
    gamma = 0.1,
    kernel = 'poly',
    degree = 3,
    coef0 = 10.0
).fit(train_ds.values.tolist(), train_label.tolist())
test_label = model.predict(test_ds)
answer = pd.DataFrame({'row_id':test['row_id'].tolist(),'open_flag':test_label})
answer.to_csv(r'answer.csv', index = False, header=True)
!kaggle competitions submit -c student-shopee-code-league-marketing-analytics -f answer.csv -m "Viet"

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBC

dataset = pd.read_csv('train.csv') 
testData = pd.read_csv('test.csv')
user = pd.read_csv('users.csv').fillna(0)
dataset.head()
user.head()

train = dataset.merge(user, on = ['user_id'])
test = testData.merge(user, on = ['user_id'])

to_replace = ['Never open', 'Never checkout', 'Never login']

train.replace(to_replace = to_replace,value = -1, inplace= True)
test.replace(to_replace = to_replace,value = -1, inplace= True)

features = [
            'subject_line_length',
            'last_open_day', 
            'last_login_day', 
            'last_checkout_day',
            'open_count_last_10_days', 
            'open_count_last_30_days',
            'open_count_last_60_days', 
            'login_count_last_10_days',
            'login_count_last_30_days', 
            'login_count_last_60_days',
            'checkout_count_last_10_days', 
            'checkout_count_last_30_days',
            'checkout_count_last_60_days', 
            'attr_1',
            'attr_2', 
            'attr_3'
            ]

train_ds = train[features]
train_label = train['open_flag']
test_ds = test[features]

model = GBC(
    loss = 'deviance',
    learning_rate = 0.1,
    n_estimator = 500,
    max_depth = 3,
    max_features = 'log2'
).fit(train_ds.values.tolist(), train_label.tolist())
test_label = model.predict(test_ds)
answer = pd.DataFrame({'row_id':test['row_id'].tolist(),'open_flag':test_label})
answer.to_csv(r'answer.csv', index = False, header=True)
!kaggle competitions submit -c student-shopee-code-league-marketing-analytics -f answer.csv -m "GBC"

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0
!pip install thundersvm
!pip install thundersvm-cpu

--2021-02-16 02:39:24--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2021-02-16 02:39:24--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?XLb9jkHi9cSHrou50_XonwZibJveauqxLHqxbAAf9OMpDYW6y5pYUU_Hz3AdPUSAaIYdpmpktLPNF4Lz6XDJHRNGtzXvA-nzD

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from thundersvm import SVC

dataset = pd.read_csv('train.csv') 
testData = pd.read_csv('test.csv')
user = pd.read_csv('users.csv').fillna(0)
dataset.head()
user.head()

train = dataset.merge(user, on = ['user_id'])
test = testData.merge(user, on = ['user_id'])

to_replace = ['Never open', 'Never checkout', 'Never login']

train.replace(to_replace = to_replace,value = -1, inplace= True)
test.replace(to_replace = to_replace,value = -1, inplace= True)

features = [
            'subject_line_length',
            'last_open_day', 
            'last_login_day', 
            'last_checkout_day',
            'open_count_last_10_days', 
            'open_count_last_30_days',
            'open_count_last_60_days', 
            'login_count_last_10_days',
            'login_count_last_30_days', 
            'login_count_last_60_days',
            'checkout_count_last_10_days', 
            'checkout_count_last_30_days',
            'checkout_count_last_60_days', 
            'attr_1',
            'attr_2', 
            'attr_3'
            ]

train_ds = train[features]
train_label = train['open_flag']
test_ds = test[features]

model = SVC()
model.fit(train_ds.values.tolist(), train_label.tolist())
test_label = model.predict(test_ds)
answer = pd.DataFrame({'row_id':test['row_id'].tolist(),'open_flag':test_label})
answer.to_csv(r'answer.csv', index = False, header=True)
!kaggle competitions submit -c student-shopee-code-league-marketing-analytics -f answer.csv -m "ThunderSVC"

In [2]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CUDA_HOME/lib64/:$CUDA_HOME/lib/:$CUDA_HOME/extras/CUPTI/lib64
!pip install -U --force-reinstall ipykernel
!pip install parallel
!pip install --force-reinstall -I h2o4gpu
!pip install TPOT

     |████████████████████████████████| 120 kB 8.9 MB/s 
     |████████████████████████████████| 108 kB 27.3 MB/s 
     |████████████████████████████████| 427 kB 23.0 MB/s 
     |████████████████████████████████| 75 kB 6.1 MB/s 
     |████████████████████████████████| 785 kB 37.3 MB/s 
     |████████████████████████████████| 784 kB 44.1 MB/s 
     |████████████████████████████████| 983 kB 72.7 MB/s 
     |████████████████████████████████| 59 kB 8.1 MB/s 
     |████████████████████████████████| 366 kB 74.0 MB/s 
     |████████████████████████████████| 1.4 MB 68.9 MB/s 
     |████████████████████████████████| 93 kB 2.7 MB/s 
     |████████████████████████████████| 82 kB 1.4 MB/s 
     |████████████████████████████████| 227 kB 76.9 MB/s 
     |████████████████████████████████| 1.1 MB 75.3 MB/s 
  Attempting uninstall: six
    Found existing installation: six 1.15.0
    Uninstalling six-1.15.0:
      Successfully uninstalled six-1.15.0
  Attempting uninstall: ipython-genutils
    Found exi

     |████████████████████████████████| 57 kB 4.4 MB/s 
     |████████████████████████████████| 48 kB 4.8 MB/s 
     |████████████████████████████████| 48 kB 4.7 MB/s 
ERROR: Could not find a version that satisfies the requirement parallel
ERROR: No matching distribution found for parallel
     |████████████████████████████████| 346.3 MB 27 kB/s 
     |████████████████████████████████| 9.5 MB 73.2 MB/s 
     |████████████████████████████████| 14.8 MB 172 kB/s 
  Using cached python_dateutil-2.8.1-py2.py3-none-any.whl (227 kB)
     |████████████████████████████████| 510 kB 79.5 MB/s 
     |████████████████████████████████| 25.9 MB 37 kB/s 
     |████████████████████████████████| 6.7 MB 29.0 MB/s 
     |████████████████████████████████| 303 kB 81.5 MB/s 
     |████████████████████████████████| 829 kB 85.8 MB/s 
     |████████████████████████████████| 291 kB 71.8 MB/s 
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.wh

     |████████████████████████████████| 87 kB 4.3 MB/s 
     |████████████████████████████████| 157 kB 15.8 MB/s 
     |████████████████████████████████| 157.5 MB 59 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11955 sha256=ebdcb39386c018f2c0b3363d668b806508d0424ab9fbd9a4912da63b2f84a5d2
  Stored in directory: /root/.cache/pip/wheels/07/2e/ce/e558b7d4f9aafcdc0e5638ef890a3d5166d8a0f2c2dc768379
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBC

dataset = pd.read_csv('train.csv') 
testData = pd.read_csv('test.csv')
user = pd.read_csv('users.csv').fillna(0)
dataset.head()
user.head()

train = dataset.merge(user, on = ['user_id'])
test = testData.merge(user, on = ['user_id'])

to_replace = ['Never open', 'Never checkout', 'Never login']

train.replace(to_replace = to_replace,value = -1, inplace= True)
test.replace(to_replace = to_replace,value = -1, inplace= True)

features = [
            'subject_line_length',
            'last_open_day', 
            'last_login_day', 
            'last_checkout_day',
#            'open_count_last_10_days', 
            'open_count_last_30_days',
#           'open_count_last_60_days', 
#            'login_count_last_10_days',
            'login_count_last_30_days', 
#            'login_count_last_60_days',
#            'checkout_count_last_10_days', 
            'checkout_count_last_30_days',
#            'checkout_count_last_60_days', 
            'attr_1',
            'attr_2', 
            'attr_3'
            ]

train_ds = train[features]
train_label = train['open_flag']
test_ds = test[features]

model = GBC(
    loss = 'deviance',
    learning_rate = 0.1,
    n_estimators = 500,
    max_depth = 3,
    max_features = 'log2'
).fit(train_ds.values.tolist(), train_label.tolist())
test_label = model.predict(test_ds)
answer = pd.DataFrame({'row_id':test['row_id'].tolist(),'open_flag':test_label})
answer.to_csv(r'answer.csv', index = False, header=True)
!kaggle competitions submit -c student-shopee-code-league-marketing-analytics -f answer.csv -m "GBC"

100% 426k/426k [00:02<00:00, 191kB/s]
Successfully submitted to [Student] Shopee Code League - Marketing Analytics